<a href="https://colab.research.google.com/github/stephvp1172/DeepLearningMidterm/blob/main/DeepLearningMidterm_StephaniePalanca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Statistical Deep Learning Midterm - Identifying Fashion Pieces using the Fashion MNIST Dataset
##### Stephanie Palanca

In this midterm submission, I will be creating CNN and ANN models and comparing their performances in identifying fashion pieces from the Fashion MNIST dataset.

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
from keras.datasets import fashion_mnist

#### Simple Dense Neural Network
Using the ReLU activation function, with Data Augmentation, Batch Size of 512, 10 classes, and 20 predictions

In [ ]:
batch_size = 512
num_classes = 10
epochs = 30
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_fashion_mnsit_trained_model.h5'

In [ ]:
#loading dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [ ]:
#printing training data shape, # of train and test samples
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
#simple dense neural network
def simple_dense_model(num_hidden_layer):
    model = Sequential()
    model.add(Flatten(input_shape=x_train.shape[1:]))

    for _ in range(num_hidden_layer):
        model.add(Dense(512))
        model.add(Activation('relu'))
        model.add(Dropout(0.5))

    # Output layer with 10 units (for 10 classes)
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    return model

In [ ]:
#Training the CNN

#defining model variable
model = simple_dense_model(num_hidden_layer=0)

#compiling model
model.compile(loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy'])

# Train the model
history_dnn = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        shuffle=True)

# Train and evaluate dense neural networks with 0-4 hidden layers
num_hidden_layers_list = [0, 1, 2, 3, 4]
history_dense_models = []

for num_hidden_layers in num_hidden_layers_list:
    dense_model = simple_dense_model(num_hidden_layers)
    dense_model.compile(loss='categorical_crossentropy',
                        optimizer='adam',
                        metrics=['accuracy'])

    history = dense_model.fit(x_train, y_train,
                              batch_size=batch_size,
                              epochs=epochs,
                              validation_data=(x_test, y_test),
                              shuffle=True,
                              verbose=0)  # Set verbose to 0 to suppress output

    history_dense_models.append(history)

In [ ]:
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
#visualizing training accuracy
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.xlabel('Epochs')
plt.ylabel('Training Accuracy')
plt.title('Training Accuracy Comparison')

plt.plot(history_dnn.history['accuracy'], label='Simple Dense Neural Network', linestyle='dashed', marker='o')
for i, num_hidden_layers in enumerate(num_hidden_layers_list):
    plt.plot(history_dense_models[i].history['accuracy'], label=f'Dense {num_hidden_layers} layers')

plt.legend()
plt.grid()
plt.show()

In [ ]:
#visualizing Validation accuracy scores
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.xlabel('Epochs')
plt.ylabel('Validation Accuracy')
plt.title('Validation Accuracy Comparison')

plt.plot(history_dnn.history['val_accuracy'], label='Simple Dense Neural Network', linestyle='dashed', marker='o')
for i, num_hidden_layers in enumerate(num_hidden_layers_list):
    plt.plot(history_dense_models[i].history['val_accuracy'], label=f'Dense {num_hidden_layers} layers')

plt.legend()
plt.grid()
plt.show()

In [ ]:
#Prediction Example

#predicting test data
predictions = model.predict(x_test)

# visualizing prediction for a specific example
example_index = 0
predicted_probabilities = predictions[example_index]
actual_label = y_test[example_index]

# Getting predicted class
predicted_class = predicted_probabilities.argmax()

# Printing Actual class/label and predicted class/label
print(f"Actual Label: {actual_label.argmax()}")
print(f"Predicted Class: {predicted_class}")

#visualizing the predicted probability for this class
import matplotlib.pyplot as plt

plt.bar(range(num_classes), predicted_probabilities)
plt.xlabel('Class')
plt.ylabel('Probability')
plt.title('Predicted Class Probabilities')
plt.xticks(range(num_classes))
plt.show()

### Convolutional Neural Network
Using the ReLU activation function, with Data Augmentation, Batch Size of 512, 10 classes, and 20 predictions

In [ ]:
from keras.layers import Conv2D, MaxPooling2D

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Ensure that the input data has the shape (batch_size, height, width, channels)
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
#defining cnn model
def simple_cnn_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    return model


In [ ]:
# Training the CNN
model = simple_cnn_model()

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
#fitting cnn
history_cnn = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(x_test, y_test),
    shuffle=True
)


In [ ]:
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
#visualizing training accuracy
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.xlabel('Epochs')
plt.ylabel('Training Accuracy')
plt.title('Training Accuracy Comparison')

plt.plot(history_cnn.history['accuracy'], label='CNN', linestyle='dashed', marker='o')
for i, num_hidden_layers in enumerate(num_hidden_layers_list):
    plt.plot(history_dense_models[i].history['accuracy'], label=f'Dense {num_hidden_layers} layers')

plt.legend()
plt.grid()
plt.show()

In [ ]:
#visualizing Validation accuracy scores
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.xlabel('Epochs')
plt.ylabel('Validation Accuracy')
plt.title('Validation Accuracy Comparison')

plt.plot(history_cnn.history['val_accuracy'], label='CNN', linestyle='dashed', marker='o')
for i, num_hidden_layers in enumerate(num_hidden_layers_list):
    plt.plot(history_dense_models[i].history['val_accuracy'], label=f'Dense {num_hidden_layers} layers')

plt.legend()
plt.grid()
plt.show()

In [ ]:
#Prediction Example

#predicting test data
predictions = model.predict(x_test)

# visualizing prediction for a specific example
predicted_probabilities = predictions[2]
actual_label = y_test[2]

# Getting predicted class
predicted_class = predicted_probabilities.argmax()

# Printing Actual class/label and predicted class/label
print(f"Actual Label: {actual_label.argmax()}")
print(f"Predicted Class: {predicted_class}")

#visualizing the predicted probability for this class
import matplotlib.pyplot as plt

plt.bar(range(num_classes), predicted_probabilities)
plt.xlabel('Class')
plt.ylabel('Probability')
plt.title('Predicted Class Probabilities (CNN)')
plt.xticks(range(num_classes))
plt.show()